In [1]:
from glob import glob
import codecs
from datetime import datetime

In [2]:
import numpy
import pandas

In [3]:
import jaydebeapi

<h3>資料樣例</h3>
<pre>
Date	Open	High	Low	Close	AdjClose	Volume
Dec 30, 2020	12,860.75	12,909.75	12,820.50	12,841.50	12,841.50	334,304
Dec 29, 2020	12,845.25	12,918.25	12,808.25	12,841.00	12,841.00	393,995
Dec 28, 2020	12,676.25	12,855.50	12,655.25	12,832.75	12,832.75	346,691
Dec 24, 2020	-	-	-	-	-	-
</pre>

<h3>環境設置</h3>

In [4]:
data_path = '/Users/justin/Desktop/資料庫管理/data/W06_data'
file_list = glob('%s/NQF-*.txt' % data_path)
file_list

['/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2017.txt',
 '/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2019.txt',
 '/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2018.txt',
 '/Users/justin/Desktop/資料庫管理/data/W06_data/NQF-2020.txt']

In [7]:
stock_ID = 'NQF'
data_table = 'STOCKINFO'

In [9]:
number_fields = [
    'StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
    'StockAdjClose', 'StockVol'
]

In [10]:
field_names = ['StockID', 'StockDate']
field_names.extend(number_fields)
field_names

['StockID',
 'StockDate',
 'StockOpen',
 'StockHigh',
 'StockLow',
 'StockClose',
 'StockAdjClose',
 'StockVol']

<h3>工具函數</h3>

In [11]:
def import_data(stockID, file_name):
    _dataset = pandas.read_csv(
        file_name, delimiter='\t', parse_dates=[0], infer_datetime_format=True,
        header=0, names=field_names[1:] #用自己的欄位名稱,從1之後開始加入(尚無stockID)
    )
    
    _dataset['StockID'] = stockID
    _dataset = _dataset.sort_values(by=['StockDate'])
    
    _dataset.reset_index(drop=True, inplace=True)
    
    _dataset = _dataset.loc[_dataset['StockVol'] != '-']
    
    _dataset[number_fields] = _dataset[number_fields].apply(
        lambda x: pandas.to_numeric(x.astype(str).str.replace(',', ''), errors='coerce')
    )
    
    _dataset['StockVol'] = _dataset['StockVol'] / 1000.0
            
    return _dataset[field_names]

In [12]:
def generate_insert_table_sql(stock_dataset, insert_table_sql_list):
    _sql = 'INSERT INTO %s ' % data_table
    _sql = _sql + '(%s, %s, %s, %s, %s, %s, %s, %s) ' % tuple(stock_dataset.columns)

    for i in range(stock_dataset.shape[0]):
        _row = stock_dataset.iloc[i]
        _insert_table_sql = _sql + "VALUES('%s','%s',%s,%s,%s,%s,%s,%s)" % (
            _row['StockID'], _row['StockDate'].strftime('%Y-%m-%d'), 
            _row['StockOpen'], _row['StockHigh'], _row['StockLow'], _row['StockClose'],
            _row['StockAdjClose'], _row['StockVol']
        )    

        insert_table_sql_list.append(_insert_table_sql)

<h3>載入檔案群，產生新增資料表格筆錄 SQL 命令稿</h3>

In [13]:
_insert_table_sql_list = []

for _file in file_list:
    stock_dataset = import_data(stock_ID, _file)
    generate_insert_table_sql(stock_dataset, _insert_table_sql_list)

insert_table_sql_list = numpy.array(_insert_table_sql_list)
#轉換成numpy.array以便後續處理


In [14]:
insert_table_sql_list.shape[0]

993

In [15]:
insert_table_sql_list[0]

"INSERT INTO STOCKINFO (StockID, StockDate, StockOpen, StockHigh, StockLow, StockClose, StockAdjClose, StockVol) VALUES('NQF','2017-01-03',4876.75,4927.5,4876.75,4906.25,4906.25,215.983)"

<h3>開啟資料庫(單人版)</h3>

In [13]:
dbConnection = jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:C:/Users/user/anaconda3/Projects/data/H2",
    ["SA", ""],
    "C:/Java/h2/bin/h2-1.4.200.jar")

dbCursor = dbConnection.cursor()
#作業空間

<h3>新增資料筆錄</h3>

In [14]:
_sql = "DELETE FROM STOCKINFO WHERE StockID='%s'" % stock_ID
dbCursor.execute(_sql)
#只要資料中有NQF的ID就先刪除

for _sql in insert_table_sql_list:
    try:
        dbCursor.execute(_sql)
    except Exception as err:
        print(err)
#一個個執行,有錯就顯示        

_sql = "SELECT COUNT(StockID) FROM STOCKINFO WHERE StockID='%s'" % stock_ID
#所有NQF的ID有幾筆,確認有正確輸入
dbCursor.execute(_sql)

resultSet = dbCursor.fetchall()

for row in resultSet:
    print(row)

(993,)


<h3>關閉資料庫</h3>

In [15]:
dbCursor.close()
dbConnection.close()